In [1]:
import json
import os
import pickle
import psycopg2
import pandas as pd
import sqlalchemy
import time
import sys
sys.path.append("..")

from connect_db import db_connection

from datetime import datetime

import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import matplotlib.cm as cm
import seaborn as sns
get_ipython().magic(u'matplotlib inline')
%matplotlib inline

username='ywang99587'

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler



from scipy.spatial import distance
import scipy.spatial.distance
from scipy.spatial.distance import cdist, pdist
import pylab as pl

In [2]:
cred_location = '/mnt/data/ywang99587/utils/data_creds_redshift.json.nogit'  
db = db_connection.DBConnection(cred_location)

In [3]:
def query_mcc_season(mcc,
                          min_date = '2017-05-01 00:00:00', 
                          max_date = '2018-02-28 23:59:59'):
    '''
    Run the query for customer features of a given country in a given time window
    
    Params:
        mcc: mobile country code
        min_date: minimum date, in the yyy-mm-dd hh:mm:ss format
        max_date: maximum date, in the same format
        
    Returns:
        df_mcc_season: feature table of a given country in a given season
    '''
        
    query = f'with rel_customer as (select distinct customer_id from tpt.tuscany.vodafone where mcc={mcc} \
    and (time_stamp between {min_date!r} and {max_date!r}) \
    and customer_id not in (select customer_id from tuscany.excluded_customers)) \
    select * from tpt.tuscany.customer_feature \
    where customer_id in (select customer_id from rel_customer)'
    df_mcc_season = db.sql_query_to_data_frame(query, cust_id=True)
    
    
    return df_mcc_season

In [4]:
mcc_ls = [262, 310, 208]

In [5]:
start_dt_GER = [str(datetime(2017, 5, 1, hour=0, minute=0, second=0)), 
                str(datetime(2017, 6, 1, hour=0, minute=0, second=0)),
                str(datetime(2017, 9, 1, hour=0, minute=0, second=0)), 
                str(datetime(2017, 11, 1, hour=0, minute=0, second=0))]

In [6]:
end_dt_GER = [str(datetime(2017, 5, 31, hour=23, minute=59, second=59)),
              str(datetime(2017, 8, 31, hour=23, minute=59, second=59)),
              str(datetime(2017, 10, 31, hour=23, minute=59, second=59)),
              str(datetime(2018, 2, 28, hour=23, minute=59, second=59))]

In [22]:
df_GER = []
mcc = 262
for s in range(3,len(start_dt_GER)):
    df = query_mcc_season(mcc=mcc, min_date = start_dt_GER[s], max_date = end_dt_GER[s])
    df_GER.append(df)
    df.to_csv(f'/mnt/data/shared/{"ger"+str(s)+".csv"}')

In [23]:
df.head()

,mcc,hr_arvl_tusc,day_of_wk_arvl_tusc,mon_arvl_tusc,day_arvl_tusc,loc_arvl_tusc,hr_arvl_italy,day_of_wk_arvl_italy,mon_arvl_italy,day_arvl_italy,...,std_lon,hrs_outside_tuscany,start_lat_tusc,start_lon_tusc,num_unique_loc_in_tusc,num_loc_in_tusc,num_unique_loc_in_italy,num_loc_in_italy,st_time,en_time
0,262,16,1,10,30,57340012,12,1,10,30,...,0.076676,7.00,43.9908,11.2144,27,1450,57,1549,2017-10-30 12:50:00,2017-11-06 13:25:00
1,262,12,3,11,15,3629073,12,3,11,15,...,0.176967,0.00,43.6185,10.2917,69,157,124,1093,2017-11-15 12:50:00,2017-11-17 13:12:00
2,262,19,6,10,28,3626601,13,6,10,28,...,0.410009,11.00,43.9159,11.2078,354,1189,517,1484,2017-10-28 13:18:00,2017-11-02 23:20:00
3,262,15,2,11,21,57340012,7,2,11,21,...,0.150742,13.00,43.9908,11.2144,180,1505,379,1851,2017-11-21 07:02:00,2017-11-25 23:58:00
4,262,12,1,10,30,3621376,8,1,10,30,...,1.596464,5.00,43.9954,11.3653,44,50,297,1006,2017-10-30 08:26:00,2017-11-03 11:28:00


In [16]:
print(f'/mnt/data/shared/{"ger"+str(4)+".csv"}')

/mnt/data/shared/ger4.csv


In [4]:
min_date = str(datetime(2017, 4, 30, hour=23, minute=59, second=59))
max_date = str(datetime(2017, 5, 31, hour=23, minute=59, second=59))
mcc = 310

a = f'with rel_customer as (select distinct customer_id from tpt.tuscany.vodafone where mcc={mcc} \
    and (time_stamp between {min_date!r} and {max_date!r}) \
    and customer_id not in (select customer_id from tuscany.excluded_customers)) \
    select * from tpt.tuscany.customer_feature \
    where customer_id in (select customer_id from rel_customer)'
print(a)

with rel_customer as (select distinct customer_id from tpt.tuscany.vodafone where mcc=310     and (time_stamp between '2017-04-30 23:59:59' and '2017-05-31 23:59:59')     and customer_id not in (select customer_id from tuscany.excluded_customers))     select * from tpt.tuscany.customer_feature     where customer_id in (select customer_id from rel_customer)


In [29]:
start_dt = str(datetime(2017, 4, 30, hour=23, minute=59, second=59))
end_dt = str(datetime(2017, 5, 31, hour=23, minute=59, second=59))
mcc = 310
df_US_May = query_mcc_season(mcc=mcc, min_date = start_dt, max_date = end_dt)

In [30]:
df_US_May.head()

,mcc,hr_arvl_tusc,day_of_wk_arvl_tusc,mon_arvl_tusc,day_arvl_tusc,loc_arvl_tusc,hr_arvl_italy,day_of_wk_arvl_italy,mon_arvl_italy,day_arvl_italy,...,std_lon,hrs_outside_tuscany,start_lat_tusc,start_lon_tusc,num_unique_loc_in_tusc,num_loc_in_tusc,num_unique_loc_in_italy,num_loc_in_italy,st_time,en_time
0,310,11,2,5,2,3615752,0,2,5,2,...,1.592567,10.00,43.0114,11.9393,191,488,330,766,2017-05-02 00:25:00,2017-05-13 19:09:00
1,310,20,1,5,1,59527011,0,1,5,1,...,0.737204,20.00,42.8047,10.3146,3,10,22,91,2017-05-01 00:02:00,2017-05-02 18:41:00
2,310,1,1,5,1,3653233,1,1,5,1,...,1.007917,10.00,43.8827,10.7693,89,316,237,628,2017-05-01 01:11:00,2017-05-03 23:57:00
3,310,0,1,5,1,59907032,0,1,5,1,...,0.610004,17.00,43.7816,11.2340,87,231,112,287,2017-05-01 00:03:00,2017-05-04 23:03:00
4,310,13,1,5,1,3615851,5,1,5,1,...,1.255647,16.00,44.1192,11.4353,35,61,248,728,2017-05-01 05:42:00,2017-05-03 23:52:00


In [29]:
df_US_May.shape

(726168, 35)

In [3]:
query_feature = """select *
        from tpt.tuscany.customer_feature
        where customer_id not in (select customer_id from tuscany.excluded_customers)"""  #mon_arvl_italy <= 8 and mon_arvl_italy >= 7 and 

In [4]:
df_feature_all = db.sql_query_to_data_frame(query_feature, cust_id=True)


In [5]:
df_feature_all.head()

,mcc,hr_arvl_tusc,day_of_wk_arvl_tusc,mon_arvl_tusc,day_arvl_tusc,loc_arvl_tusc,hr_arvl_italy,day_of_wk_arvl_italy,mon_arvl_italy,day_arvl_italy,...,std_lon,hrs_outside_tuscany,start_lat_tusc,start_lon_tusc,num_unique_loc_in_tusc,num_loc_in_tusc,num_unique_loc_in_italy,num_loc_in_italy,st_time,en_time
0,204,0,1,5,1,3654673,0,1,5,1,...,0.608433,3.00,42.9833,11.8750,42,246,71,280,2017-05-01 00:21:00,2017-05-06 11:45:00
1,232,16,1,5,1,3631302,7,1,5,1,...,0.535149,18.00,44.0757,11.2259,56,86,138,263,2017-05-01 07:01:00,2017-05-06 19:33:00
2,460,15,1,5,1,3615552,11,1,5,1,...,0.348886,5.00,43.0889,11.8882,122,275,145,323,2017-05-01 11:23:00,2017-05-04 09:47:00
3,400,12,1,5,1,3048792,10,1,5,1,...,0.336967,15.00,44.1361,11.4643,80,222,90,255,2017-05-01 10:31:00,2017-05-07 23:36:00
4,232,0,1,5,1,3041311,0,1,5,1,...,1.584984,0.00,43.8875,10.7723,165,540,237,668,2017-05-01 00:00:00,2017-05-06 09:42:00


In [13]:
df_feature_all.shape

(6194002, 2)

In [7]:
df_feature_all[df_feature_all.columns]=df_feature_all[df_feature_all.columns].astype(float)

In [7]:
df_feature_all.columns

Index(['mcc', 'hr_arvl_tusc', 'day_of_wk_arvl_tusc', 'mon_arvl_tusc',
       'day_arvl_tusc', 'loc_arvl_tusc', 'hr_arvl_italy',
       'day_of_wk_arvl_italy', 'mon_arvl_italy', 'day_arvl_italy',
       'loc_arvl_italy', 'locs_outside_tuscany', 'num_uniuqe_loc_in_italy',
       'num_loc_in_italy', 'num_uniuqe_loc_in_tusc', 'num_loc_in_tusc',
       'unique_locs_outside_tuscany', 'customer_nr', 'start_lat', 'start_lon',
       'end_lat', 'end_lon', 'hrs_in_italy', 'hrs_in_tusc', 'avg_lat',
       'avg_lon', 'top_lat', 'top_lon', 'std_lat', 'std_lon',
       'hrs_outside_tuscany'],
      dtype='object')

In [15]:
df_feature_all.describe()

,mcc,hr_arvl_tusc,day_of_wk_arvl_tusc,mon_arvl_tusc,day_arvl_tusc,loc_arvl_tusc,hr_arvl_italy,day_of_wk_arvl_italy,mon_arvl_italy,day_arvl_italy,...,end_lon,hrs_in_italy,hrs_in_tusc,avg_lat,avg_lon,top_lat,top_lon,std_lat,std_lon,hrs_outside_tuscany
count,2127985,2127985,2127985,2127985,2127985,2127985,2127985,2127985,2127985,2127985,...,2127985,2127985,2127985,2127985,2127985,2127985,2127985,2072740,2072740,2127985
unique,221,24,7,2,31,16517,24,7,2,31,...,17354,743,720,1256202,1379605,15950,16696,1027827,1032973,48
top,262,11,6,7,5,3631302,0,6,7,5,...,11.254000,71.00,2.00,43.771400,11.254000,43.771400,11.254000,0.000000,0.000000,0.00
freq,326228,190646,456221,1101272,90703,79169,361005,456221,1101272,90703,...,32559,23952,25629,5858,5790,55026,55026,45949,45460,890856


In [ ]:
#df_feature_all['num_loc_per_hr_tusc'] = df_feature_all['num_loc_in_tusc']/df_feature_all['hrs_in_tusc']

In [ ]:
#df_feature_all2 = df_feature_all[(df_feature_all['num_loc_per_hr_tusc']>0.04) & (df_feature_all['num_loc_per_hr_tusc']<100)]

In [ ]:
## skip mcc for now

In [ ]:
mcc_region=pd.read_csv('/mnt/data/shared/mcc_country_region.csv')

In [ ]:
mcc_country_region=mcc_region.drop_duplicates("MCC", keep='first')[['MCC', 'Country','Region']]

In [ ]:
mcc_country_region['mcc']=mcc_country_region['MCC'].astype(int)

In [ ]:
def clean_guam(row):
    if row['Country']=='Guam':
        return 'United States'
    else:
        return row['Country']

In [ ]:
mcc_country_region['Country']=mcc_country_region.apply(clean_guam,1)
mcc_country_region.head()

In [ ]:
df_feature_all = pd.merge(df_feature_all, mcc_country_region[['mcc','Country','Region']], on=['mcc', 'mcc'])


In [ ]:
top20_countries = ['Germany', 'United States','France','United Kingdom','Netherlands','Spain','Switzerland','China','Belgium','Poland','Romania','Russian Federation','Canada','Austria','Korea S Republic of','Sweden','Australia','Denmark','Japan','Brazil']

In [ ]:
def top_or_region(row):
    if row['Country'] in top20_countries:
        return row['Country']
    else:
        return row['Region']

In [ ]:
df_feature_all['Country_region']=df_feature_all.apply(top_or_region,1)
df_feature_all.head()

In [ ]:
mcc_fac = pd.get_dummies(df_feature_all['Country_region']) ## turn country_region instead of mcc to dummies


In [ ]:
mcc_fac.columns

In [ ]:
df_feature_all_mcc = df_feature_all.join(mcc_fac)
df_feature_all_mcc.head()

In [ ]:

## skip the mcc first

In [ ]:
df_feature_mcc = scaled_feature.join(mcc_fac)

In [5]:
## standardize all continuous features
df_feature_cont = df_feature_all[['hrs_in_italy', 'hr_arvl_italy', 'day_of_wk_arvl_italy', 'mon_arvl_italy', 'day_arvl_italy',
       'num_uniuqe_loc_in_italy', 'num_loc_in_italy','num_uniuqe_loc_in_tusc', 'num_loc_in_tusc', 'hrs_in_tusc',
       'hr_arvl_tusc', 'day_of_wk_arvl_tusc', 'mon_arvl_tusc', 'day_arvl_tusc',
       'hrs_outside_tuscany', 'locs_outside_tuscany', 'unique_locs_outside_tuscany',
        'start_lat', 'start_lon',
       'end_lat', 'end_lon',  'avg_lat',
       'avg_lon', 'top_lat', 'top_lon' ]]
#df_feature_mcc = df_feature.join(mcc_fac)
scaler = StandardScaler()
scaled_feature_all = pd.DataFrame(scaler.fit_transform(df_feature_cont), columns = df_feature_cont.columns)
scaled_feature_all.head()

,hrs_in_italy,hr_arvl_italy,day_of_wk_arvl_italy,mon_arvl_italy,day_arvl_italy,num_uniuqe_loc_in_italy,num_loc_in_italy,num_uniuqe_loc_in_tusc,num_loc_in_tusc,hrs_in_tusc,...,locs_outside_tuscany,unique_locs_outside_tuscany,start_lat,start_lon,end_lat,end_lon,avg_lat,avg_lon,top_lat,top_lon
0,0.472303,-0.568708,1.335322,-0.965555,-1.627884,-0.748115,-0.509448,-0.689637,-0.441014,0.480580,...,-0.396288,-0.607529,-0.806519,1.030899,-0.180101,0.571005,0.057807,0.818283,0.184376,0.515941
1,0.024695,0.222978,1.335322,-0.965555,-1.627884,1.093966,1.756606,2.322064,2.134938,0.066926,...,-0.348873,-0.428209,-0.109975,0.177707,-0.018162,0.154258,0.580098,-0.384084,0.488727,-0.309243
2,0.051823,0.539652,1.335322,-0.965555,-1.627884,2.436792,1.674987,2.322064,1.652690,0.060145,...,0.736922,1.902946,1.394031,-1.255414,1.606272,-1.355551,0.035970,-0.381849,-0.002128,-0.492833
3,0.038259,0.222978,1.335322,-0.965555,-1.627884,-0.025055,0.070793,0.622391,0.251794,0.080489,...,-0.476893,-0.667302,-0.184889,-0.354555,-0.098771,-0.403735,0.459032,0.358616,0.372199,0.351783
4,2.113532,-0.568708,1.335322,-0.965555,-1.627884,3.125420,1.857518,3.842824,1.970227,2.128415,...,0.433468,1.574193,1.353859,-1.266747,-0.262231,0.442479,0.686151,-0.112552,0.417634,0.131694


In [6]:
## relevant features
scaled_feature = scaled_feature_all[['unique_locs_outside_tuscany', 'locs_outside_tuscany','num_uniuqe_loc_in_tusc', 
                                     'num_loc_in_tusc', 'hrs_in_tusc','hrs_outside_tuscany',
                                    'start_lat', 'start_lon',
       'end_lat', 'end_lon',  'avg_lat',
       'avg_lon', 'top_lat', 'top_lon']]
scaled_feature.corr()

,unique_locs_outside_tuscany,locs_outside_tuscany,num_uniuqe_loc_in_tusc,num_loc_in_tusc,hrs_in_tusc,hrs_outside_tuscany,start_lat,start_lon,end_lat,end_lon,avg_lat,avg_lon,top_lat,top_lon
unique_locs_outside_tuscany,1.000000,0.761844,0.503440,0.316790,0.249635,0.476558,0.296409,-0.105332,0.208567,-0.063747,-0.157809,0.207896,-0.170587,0.200193
locs_outside_tuscany,0.761844,1.000000,0.384595,0.252774,0.190736,0.388428,0.161691,-0.026657,0.111590,-0.011678,-0.086902,0.161009,-0.095978,0.151647
num_uniuqe_loc_in_tusc,0.503440,0.384595,1.000000,0.727344,0.303737,0.280988,0.226570,-0.126930,0.164319,-0.068688,0.055221,-0.031725,0.033366,-0.021869
num_loc_in_tusc,0.316790,0.252774,0.727344,1.000000,0.295240,0.144624,0.185156,-0.115926,0.128609,-0.061859,0.059522,-0.072180,0.046958,-0.062210
hrs_in_tusc,0.249635,0.190736,0.303737,0.295240,1.000000,-0.027338,0.183823,-0.117811,0.116689,-0.103851,-0.141602,0.072047,-0.112210,0.068616
hrs_outside_tuscany,0.476558,0.388428,0.280988,0.144624,-0.027338,1.000000,0.150825,0.093506,0.131861,0.087831,-0.033648,0.151444,-0.057906,0.127625
start_lat,0.296409,0.161691,0.226570,0.185156,0.183823,0.150825,1.000000,-0.418352,0.256190,-0.102426,0.136409,-0.111659,0.040311,-0.065613
start_lon,-0.105332,-0.026657,-0.126930,-0.115926,-0.117811,0.093506,-0.418352,1.000000,-0.106744,0.309073,-0.080942,0.367919,-0.032284,0.268012
end_lat,0.208567,0.111590,0.164319,0.128609,0.116689,0.131861,0.256190,-0.106744,1.000000,-0.463048,0.036901,-0.043154,-0.046306,-0.001518
end_lon,-0.063747,-0.011678,-0.068688,-0.061859,-0.103851,0.087831,-0.102426,0.309073,-0.463048,1.000000,-0.016740,0.202277,0.025691,0.120620


In [7]:
## scaled features and lon lat
df_scaled_loc = pd.concat([scaled_feature.reset_index(drop=True), df_feature_all[['std_lat', 'std_lon']]], axis=1)
df_scaled_loc.head()

,unique_locs_outside_tuscany,locs_outside_tuscany,num_uniuqe_loc_in_tusc,num_loc_in_tusc,hrs_in_tusc,hrs_outside_tuscany,start_lat,start_lon,end_lat,end_lon,avg_lat,avg_lon,top_lat,top_lon,std_lat,std_lon
0,-0.607529,-0.396288,-0.689637,-0.441014,0.480580,-0.165077,-0.806519,1.030899,-0.180101,0.571005,0.057807,0.818283,0.184376,0.515941,0.642902,0.948602
1,-0.428209,-0.348873,2.322064,2.134938,0.066926,-0.837219,-0.109975,0.177707,-0.018162,0.154258,0.580098,-0.384084,0.488727,-0.309243,0.088089,0.231396
2,1.902946,0.736922,2.322064,1.652690,0.060145,-0.165077,1.394031,-1.255414,1.606272,-1.355551,0.035970,-0.381849,-0.002128,-0.492833,0.654711,0.604870
3,-0.667302,-0.476893,0.622391,0.251794,0.080489,-0.837219,-0.184889,-0.354555,-0.098771,-0.403735,0.459032,0.358616,0.372199,0.351783,0.065274,0.390908
4,1.574193,0.433468,3.842824,1.970227,2.128415,-0.299506,1.353859,-1.266747,-0.262231,0.442479,0.686151,-0.112552,0.417634,0.131694,0.501544,0.608254


In [ ]:
## skip PCA now


In [ ]:
pca=PCA(n_components=6)
#pca.fit(df_feature_mcc)
pca.fit(scaled_feature) # w/o mcc
print(pca.explained_variance_ratio_) 


In [ ]:
plt.plot(range(1,7),pca.explained_variance_ratio_)
#plt.axvline(7)

In [ ]:
pca.explained_variance_ratio_.cumsum()

In [ ]:
pca.explained_variance_

In [ ]:
reduced_feature = PCA(n_components=3).fit_transform(df_feature_mcc)
#kmeans = KMeans(init='k-means++', n_clusters=2, n_init=10)
#output=kmeans.fit(reduced_feature)

In [8]:
data=df_scaled_loc.loc[df_scaled_loc['std_lat']>0]

In [ ]:
## kmeans with only two features
## use distortion?

distortions = []
sl_score = []
ch_score = []
K = range(1,10)
for k in K:
    kmeans = KMeans(n_clusters=k) ## check kmeanModel.predict?
    kmeans.fit(data)
    kmeanModel = KMeans(n_clusters=k).fit(data)
    kmeanModel.fit(data)
    distortions.append(sum(np.min(cdist(data, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / data.shape[0])
    #labels=kmeanModel.predict(reduced_feature)
    #sl_score.append(metrics.silhouette_score(reduced_feature, labels, metric='euclidean'))
    #ch_score.append(metrics.calinski_harabaz_score(reduced_feature, labels))


In [ ]:
distortions

In [ ]:
# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [50]:
## skip this
#Nc = range(2,10)

#kmeans = [KMeans(n_clusters=i, n_jobs = -1) for i in Nc]
#[model.fit(data) for model in kmeans]
#labels = [model.predict(data) for model in kmeans]

In [ ]:
sl_score = [metrics.silhouette_score(reduced_feature, labels[i], metric='euclidean', sample_size=600000) for i in range(1, 13)]


In [13]:
ch_score = [metrics.calinski_harabaz_score(data, labels[i]) for i in range(1,3)]

#plt.axvline(2)

In [9]:
ch_score = []
sl_score = []
for i in range(2,83,5):
    kmeans_model = KMeans(n_clusters=i, n_jobs = -1).fit(data)
    labels = kmeans_model.labels_
    ch = metrics.calinski_harabaz_score(data, labels)
   # sl = metrics.silhouette_score(data, labels, metric='euclidean', sample_size=80000)
    ch_score.append(ch)
   # sl_score.append(sl)
    print(ch)

371966.12571551267
252756.04915341828
198589.71954866635
166069.5337172294
145608.1415203084
131140.10724178905
119957.83016959277
111266.46855373253
103818.09820614419
97724.6735399761
92163.20729966533
87733.65310783019
83601.82991122821
79781.6133515971
76572.85878967911
73431.56751106086


Process ForkPoolWorker-260:
Process ForkPoolWorker-258:
Process ForkPoolWorker-259:
Process ForkPoolWorker-273:
Process ForkPoolWorker-271:
Process ForkPoolWorker-263:
Process ForkPoolWorker-262:
Process ForkPoolWorker-272:
Process ForkPoolWorker-264:
Process ForkPoolWorker-261:
Process ForkPoolWorker-265:
Process ForkPoolWorker-270:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ywang99587/.conda/envs/tptyw/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ywang99587/.conda/envs/tptyw/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ywang99587/.con

  File "/home/ywang99587/.conda/envs/tptyw/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ywang99587/.conda/envs/tptyw/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ywang99587/.conda/envs/tptyw/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Traceback (most recent call last):
  File "/home/ywang99587/.conda/envs/tptyw/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ywang99587/.conda/envs/tptyw/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ywang99587/.conda/envs/tptyw/lib/python3.6/site-packages/sklearn/externals/joblib/pool.py", line 360, in get
    racquire()
  File "/home/ywang99587/.conda/envs/tptyw/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home

KeyboardInterrupt: 

In [ ]:
pl.plot(list(range(2,83,5), ch_score)

In [ ]:
sns.pairplot()

In [19]:
kmeans = KMeans(n_clusters=2, n_jobs=-1)
kmeans.fit(data)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=-1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [20]:
labels = kmeans.labels_

In [21]:
type(labels)

numpy.ndarray

In [22]:
rdf = pd.DataFrame(data)

In [34]:
rdf[rdf.columns]=rdf[rdf.columns].astype(float)

In [35]:
rdf['col']=labels.astype(str)

In [ ]:
sns.pairplot(rdf,hue='col')

In [ ]:
kmeans.cluster_centers_

In [ ]:
## reduce again???

reduced2 = PCA(n_components=2).fit_transform(reduced_feature)
kmeans = KMeans(init='k-means++', n_clusters=2, n_init=10)
output=kmeans.fit(reduced2)

In [ ]:
pca2=PCA(n_components=2)
pca2.fit_transform(reduced_feature)
pca2.explained_variance_ratio_

In [ ]:
plt.scatter(reduced2[:,0], reduced2[:,1], c=output.labels_)

In [ ]:
plt.scatter(df_feature.iloc[:,0], df_feature.iloc[:,1], c=output.labels_)
plt.ylim((0,4000))

In [ ]:
scaled_sample = scaled_feature.sample(1000)
scaled_sample.head()

In [ ]:
df_feature = StandardScaler().fit_transform(df_feature)
#db = DBSCAN().fit(df_feature, eps=0.95, min_samples=10)
db = DBSCAN(eps=0.3, min_samples=10).fit(df_feature)
core_samples = db.core_sample_indices_
labels = db.labels_

In [ ]:
## calculate distance between each component pair????



In [ ]:
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)